<a href="https://colab.research.google.com/github/JhonSegura25/Datos/blob/main/Red.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 7.0.1
    Uninstalling importlib-metadata-7.0.1:
      Successfully uninstalled importlib-metadata-7.0.1


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import joblib
import streamlit as st

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Parámetros
batch_size = 32
width_shape = 224
height_shape = 224
directory = '/content/drive/MyDrive/Univercidad/Univercidad_/DATA/'

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_generator= train_datagen.flow_from_directory(directory +'TRAIN',
                                                 target_size = (width_shape, height_shape),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 3814 images belonging to 2 classes.


In [ ]:
valid_datagen = ImageDataGenerator(rescale = 1./255)
test_set = valid_datagen.flow_from_directory(directory + 'VALID',
                                            target_size = (width_shape, height_shape),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 1732 images belonging to 2 classes.


In [ ]:
class CustomModel:
    def __init__(self, width_shape, height_shape):
        self.width_shape = width_shape
        self.height_shape = height_shape
        self.model = self.build_model()

    def build_model(self):
        base_model = VGG16(weights="imagenet", include_top=False, input_shape=(self.width_shape, self.height_shape, 3))
        x = Flatten()(base_model.output)
        x = Dense(128, activation='relu')(x)
        x = Dense(128, activation='relu')(x)
        predictions = Dense(1, activation='sigmoid')(x)
        full_model = Model(inputs=base_model.input, outputs=predictions)
        for layer in base_model.layers:
            layer.trainable = False
        full_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['acc'])
        return full_model

    def train_model(self, train_generator, test_set, epochs=10):
        history = self.model.fit_generator(
            train_generator,
            validation_data=test_set,
            workers=10,
            epochs=epochs,
        )
        return history

    def save_model(self, filename='model.pkl'):
        joblib.dump(self.model, filename)

In [ ]:
# Crear una instancia de la clase CustomModel
custom_model = CustomModel(width_shape, height_shape)

# Entrenar el modelo
history = custom_model.train_model(train_generator, test_set, epochs=5)

# Guardar el modelo
custom_model.save_model('model.pkl')

58889256/58889256 [==============================] - 0s 0us/step


<ipython-input-7-4e5e3c39c0d5>:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = self.model.fit_generator(


Epoch 1/5
120/120 [==============================] - 3106s 26s/step - loss: 0.2002 - acc: 0.9227 - val_loss: 0.5295 - val_acc: 0.7084
Epoch 2/5
120/120 [==============================] - 3029s 25s/step - loss: 0.0319 - acc: 0.9929 - val_loss: 1.3604 - val_acc: 0.5826
Epoch 3/5
120/120 [==============================] - 3095s 26s/step - loss: 0.0254 - acc: 0.9919 - val_loss: 0.8631 - val_acc: 0.6755
Epoch 4/5
120/120 [==============================] - 3078s 26s/step - loss: 0.0101 - acc: 0.9969 - val_loss: 0.6007 - val_acc: 0.7402
Epoch 5/5
120/120 [==============================] - 3031s 25s/step - loss: 0.0067 - acc: 0.9982 - val_loss: 1.3047 - val_acc: 0.6374
